# Data Prep
<font size=4 color='blue'>Project: Congressional Data Scrape and Validation</font>
***

**Project Summary:**  
The Resume of Congressional Activity has been published annually since 1947. PDF versions of this document are available for download from several US government websites, including <a href="https://senate.gov">senate.gov</a>. The primary goal of this project is to scrape the data from these documents and create a dataset that can be used for analysis.


**Notebook Scope:**  
This notebook reads formatted data from Excel and prepares the data for validation. 

**Output:**  
Scrubbed data is saved to an Excel file for further validation and analysis.

***
# Notebook Setup
***

In [1]:
# Import libraries
import os
import pandas as pd
import re

In [2]:
%%html
<!-- Prevent text wrappping in dataframe displays for a cleaner print -->
<style> .dataframe td {white-space: nowrap;}</style>

***
# Read General Data
***

In [3]:
# Create a list of files to read
raw_text_a = []
path = '../data/resumes_scrubbed/legislative_activity/'
files = os.listdir(path)
print(files)

['100_1.xlsx', '100_2.xlsx', '101_1.xlsx', '101_2.xlsx', '102_1.xlsx', '102_2.xlsx', '103_1.xlsx', '103_2.xlsx', '104_1.xlsx', '104_2.xlsx', '105_1.xlsx', '105_2.xlsx', '106_1.xlsx', '106_2.xlsx', '107_1.xlsx', '107_2.xlsx', '108_1.xlsx', '108_2.xlsx', '109_1.xlsx', '109_2.xlsx', '110_1.xlsx', '110_2.xlsx', '111_1.xlsx', '111_2.xlsx', '112_1.xlsx', '112_2.xlsx', '113_1.xlsx', '113_2.xlsx', '114_1.xlsx', '114_2.xlsx', '115_1.xlsx', '115_2.xlsx', '116_1.xlsx', '116_2.xlsx', '117_1.xlsx', '117_2.xlsx', '98_1.xlsx', '98_2.xlsx', '99_1.xlsx', '99_2.xlsx']


In [4]:
# Create a list to contain the contents of each file
leg_act_list = []

In [5]:
# Read each file and concat to the legislative activity dataframe
for file_name in files:
    file_cont_df = pd.read_excel(path + file_name, index_col=0)
    file_cont_df.index = file_cont_df.index.str.strip()
    leg_act_list.append(file_cont_df)

In [6]:
# Preview file contents
leg_act_list[0].head()

,Senate,House,Total
Congress,100,100,100
Session,1,1,1
Start Date,1987-01-06 00:00:00,1987-01-06 00:00:00,1987-01-06 00:00:00
End Date,1987-12-22 00:00:00,1987-12-22 00:00:00,1987-12-22 00:00:00
Days in session,170,169,NaN


***
## Validate Variable Names
***

In [7]:
# Define standard variables (column headings). This list will be used to validate the dataframes before merging
std_headings = ['Congress', 'Session', 'Start Date', 'End Date', 'Days in session', 'Time in session', 'Pages of proceedings',
                'Extensions of remarks', 'Public bills enacted into law', 'Private bills enacted into law', 'Bills in conference',
                'Bills through conference', 'Measures passed, total', 'Measures passed, Senate bills', 'Measures passed, House bills',
                'Measures passed, Senate joint resolutions', 'Measures passed, House joint resolutions', 
                'Measures passed, Senate concurrent resolutions', 'Measures passed, House concurrent resolutions', 
                'Measures passed, Simple resolutions', 'Measures reported, total', 'Measures reported, Senate bills',
                'Measures reported, House bills', 'Measures reported, Senate joint resolutions', 'Measures reported, House joint resolutions',
                'Measures reported, Senate concurrent resolutions', 'Measures reported, House concurrent resolutions',
                'Measures reported, Simple resolutions', 'Special reports', 'Conference reports',
                'Measures pending on calendar', 'Measures introduced, total', 'Measures introduced, Bills', 
                'Measures introduced, Joint resolutions', 'Measures introduced, Concurrent resolutions', 
                'Measures introduced, Simple resolutions', 'Quorum calls', 'Yea-and-nay votes', 'Recorded votes', 'Bills vetoed', 
                'Bills not signed', 'Vetoes overridden']

In [8]:
# Review labels from each file and flag any that do not exist in the std_headings list
errors_found = False
for df in leg_act_list:
     for label in df.index:
         if label not in std_headings:
             print(f'{df.at['Congress', 'Senate']}, {df.at['Session', 'Senate']}: {label}')
             errors_found = True
if not errors_found:
    print('No errors found.')

No errors found.


***
<font color='blue'>**Note:**</font>  
Any OCR errors found were updated directly in the Excel files.

***
## Merge Data
***

In [9]:
# Create dataframe for merged data
gen_activity_df = pd.DataFrame()

In [10]:
# Transpose the dataframe read from each file and add to the general activity dataframe
for df in leg_act_list:
    df = df.transpose().reset_index()
    gen_activity_df = pd.concat([gen_activity_df, df], ignore_index=True)

In [11]:
# Preview dataframe
gen_activity_df.head()

,index,Congress,Session,Start Date,End Date,Days in session,Time in session,Pages of proceedings,Extensions of remarks,Public bills enacted into law,...,"Measures introduced, Joint resolutions","Measures introduced, Concurrent resolutions","Measures introduced, Simple resolutions",Quorum calls,Yea-and-nay votes,Recorded votes,Bills vetoed,Vetoes overridden,Bills through conference,Bills not signed
0,Senate,100,1,1987-01-06 00:00:00,1987-12-22 00:00:00,170,"1,214 hrs., 52'",18660,NaN,96,...,239,95,353,36,420,NaN,1,2,NaN,NaN
1,House,100,1,1987-01-06 00:00:00,1987-12-22 00:00:00,169,"909 hrs., 57'",13078,NaN,144,...,437,235,345,23,234,254,2,2,NaN,NaN
2,Total,100,1,1987-01-06 00:00:00,1987-12-22 00:00:00,NaN,NaN,31738,5016,240,...,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
3,Senate,100,2,1988-01-25 00:00:00,1988-10-22 00:00:00,137,"1,126 hrs., 52'",17397,NaN,193,...,156,74,166,26,379,NaN,6,NaN,NaN,NaN
4,House,100,2,1988-01-25 00:00:00,1988-10-22 00:00:00,129,"749 hrs., 01'",11290,NaN,280,...,241,163,263,14,308,143,10,NaN,NaN,NaN


In [12]:
# Rename the Chamber column
gen_activity_df.rename(columns={'index': 'Chamber'}, inplace=True)

***
## Validate Datatypes
***

In [13]:
# Review current datatypes
gen_activity_df.dtypes

Chamber                                             object
Congress                                            object
Session                                             object
Start Date                                          object
End Date                                            object
Days in session                                     object
Time in session                                     object
Pages of proceedings                                object
Extensions of remarks                               object
Public bills enacted into law                       object
Private bills enacted into law                      object
Bills in conference                                 object
Measures passed, total                              object
Measures passed, Senate bills                       object
Measures passed, House bills                        object
Measures passed, Senate joint resolutions           object
Measures passed, House joint resolutions            obje

In [14]:
# Infer datatypes
gen_activity_df = gen_activity_df.infer_objects()

In [15]:
# Review updated datatypes
gen_activity_df.dtypes

Chamber                                                     object
Congress                                                     int64
Session                                                      int64
Start Date                                          datetime64[ns]
End Date                                            datetime64[ns]
Days in session                                            float64
Time in session                                             object
Pages of proceedings                                       float64
Extensions of remarks                                      float64
Public bills enacted into law                              float64
Private bills enacted into law                             float64
Bills in conference                                        float64
Measures passed, total                                     float64
Measures passed, Senate bills                              float64
Measures passed, House bills                               flo

***
<font color='blue'>**Note:**</font>  
Any numeric fields that did not convert to float were examined for OCR issues. These issues were corrected in the Excel files.

***
## Format Time
***

In [16]:
# Define function to format time from X hrs., Y' to HH:MM
def format_time(time_str):
    search_result = re.search(r'(\d+) hrs.[ ]?(\d+)?', time_str.replace(',', ''))
    if search_result.groups()[1] == None:
        return search_result[1]
    else:
        return search_result[1] + ':' + search_result[2]

In [17]:
# Apply format_time function to Time column
gen_activity_df['Time in session'] = gen_activity_df['Time in session'].dropna().apply(format_time)

In [18]:
# Preview dataframe
gen_activity_df.head()

,Chamber,Congress,Session,Start Date,End Date,Days in session,Time in session,Pages of proceedings,Extensions of remarks,Public bills enacted into law,...,"Measures introduced, Joint resolutions","Measures introduced, Concurrent resolutions","Measures introduced, Simple resolutions",Quorum calls,Yea-and-nay votes,Recorded votes,Bills vetoed,Vetoes overridden,Bills through conference,Bills not signed
0,Senate,100,1,1987-01-06,1987-12-22,170.0,1214:52,18660.0,NaN,96.0,...,239.0,95.0,353.0,36.0,420.0,NaN,1.0,2.0,NaN,NaN
1,House,100,1,1987-01-06,1987-12-22,169.0,909:57,13078.0,NaN,144.0,...,437.0,235.0,345.0,23.0,234.0,254.0,2.0,2.0,NaN,NaN
2,Total,100,1,1987-01-06,1987-12-22,NaN,NaN,31738.0,5016.0,240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
3,Senate,100,2,1988-01-25,1988-10-22,137.0,1126:52,17397.0,NaN,193.0,...,156.0,74.0,166.0,26.0,379.0,NaN,6.0,NaN,NaN,NaN
4,House,100,2,1988-01-25,1988-10-22,129.0,749:01,11290.0,NaN,280.0,...,241.0,163.0,263.0,14.0,308.0,143.0,10.0,NaN,NaN,NaN


***
## Sort Dataframe
***

In [19]:
# Sort data by Congress and Session and Chamber
gen_activity_df.sort_values(['Congress', 'Session', 'Chamber'], inplace=True)
gen_activity_df.reset_index(drop=True, inplace=True)
gen_activity_df.head()

,Chamber,Congress,Session,Start Date,End Date,Days in session,Time in session,Pages of proceedings,Extensions of remarks,Public bills enacted into law,...,"Measures introduced, Joint resolutions","Measures introduced, Concurrent resolutions","Measures introduced, Simple resolutions",Quorum calls,Yea-and-nay votes,Recorded votes,Bills vetoed,Vetoes overridden,Bills through conference,Bills not signed
0,House,98,1,1983-01-03,1983-11-18,146.0,851:45,10665.0,NaN,114.0,...,440.0,237.0,385.0,35.0,297.0,201.0,4.0,1.0,29.0,NaN
1,Senate,98,1,1983-01-03,1983-11-18,150.0,1010:47,17224.0,NaN,101.0,...,209.0,86.0,302.0,18.0,381.0,NaN,3.0,NaN,4.0,NaN
2,Total,98,1,1983-01-03,1983-11-18,NaN,NaN,27889.0,5985.0,215.0,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,33.0,NaN
3,House,98,2,1984-01-23,1984-10-12,120.0,852:59,12293.0,NaN,242.0,...,223.0,142.0,235.0,55.0,227.0,181.0,9.0,1.0,30.0,NaN
4,Senate,98,2,1984-01-23,1984-10-12,131.0,940:28,14650.0,NaN,166.0,...,150.0,69.0,186.0,19.0,292.0,NaN,8.0,NaN,22.0,NaN


***
## Write to Excel
***

In [20]:
gen_activity_df.to_excel('../data/general_legislative_data.xlsx', index=False)

***
# Read Confirmation Data
***

In [21]:
# Create a list of files to read
raw_text_a = []
path = '../data/resumes_scrubbed/confirmations/'
files = os.listdir(path)
print(files)

['100_1.xlsx', '100_2.xlsx', '101_1.xlsx', '101_2.xlsx', '102_1.xlsx', '102_2.xlsx', '103_1.xlsx', '103_2.xlsx', '104_1.xlsx', '104_2.xlsx', '105_1.xlsx', '105_2.xlsx', '106_1.xlsx', '106_2.xlsx', '107_1.xlsx', '107_2.xlsx', '108_1.xlsx', '108_2.xlsx', '109_1.xlsx', '109_2.xlsx', '110_1.xlsx', '110_2.xlsx', '111_1.xlsx', '111_2.xlsx', '112_1.xlsx', '112_2.xlsx', '113_1.xlsx', '113_2.xlsx', '114_1.xlsx', '114_2.xlsx', '115_1.xlsx', '115_2.xlsx', '116_1.xlsx', '116_2.xlsx', '117_1.xlsx', '117_2.xlsx', '98_1.xlsx', '98_2.xlsx', '99_1.xlsx', '99_2.xlsx']


In [22]:
# Create a list to contain the contents of each file
conf_list = []

In [23]:
# Read each file and concat to the legislative activity dataframe
for file_name in files:
    file_cont_df = pd.read_excel(path + file_name, index_col=0)
    file_cont_df.index = file_cont_df.index.str.strip()
    conf_list.append(file_cont_df)

In [24]:
# Preview file contents
conf_list[0].head()

,Values
Labels,
Congress,100
Session,1
Start Date,1987-01-06 00:00:00
End Date,1987-12-22 00:00:00
Civilian,NaN


***
## Validate Variable Names
***

In [25]:
# Define standard variables (column headings). This list will be used to validate the dataframes before merging
std_headings = ['Congress', 'End Date', 'Session', 'Start Date', 'Total carryover nominations', 'Total confirmed', 
                'Total failed at Aug adjournment', 'Total failed at August-September adjournment', 
                'Total failed at November sine die adjournment', 'Total failed at adjournment', 
                'Total failed at sine die adjournment', 'Total new nominations', 'Total recess reappointments', 
                'Total rejected', 'Total returned', 'Total returned at sine die adjournment', 
                'Total returned to the White House', 'Total superseded by recess reappointments', 'Total unconfirmed', 'Total withdrawn']
branches = ['Air Force', 'Army', 'Civilian', 'Civilian (FS, PHS, CG, NOAA)', 'Civilian (lists)', 'Civilian (other than lists)', 
            'Civilian lists', 'Marine Corps', 'Navy', 'Other Civilian', 'Other Civilian (lists)', 'Space Force']
labels = ['Carryover nominations', 'Confirmed', 'Failed at Aug adjournment', 'Failed at August-September adjournment', 
          'Failed at November sine die adjournment', 'Failed at adjournment', 'Failed at sine die adjournment', 'New nominations', 
          'Recess reappointments', 'Rejected', 'Returned', 'Returned at sine die adjournment', 'Returned to White House', 
          'Superseded by recess reappointments', 'Unconfirmed', 'Withdrawn']

In [26]:
# Review labels from each file and flag any that do not exist in the std_headings list
errors_found = False
for df in conf_list:
    for item in df.index:
        if ',' in item and item[-1:] != ')':
            branch, label = item.rsplit(', ', maxsplit=1)
            if branch not in branches or label not in labels:
                print(f'Missing branch or label -- {df.at["Congress", "Values"]}, {df.at["Session", "Values"]}: {branch}, {label}')
                errors_found = True
        else:
            if item not in std_headings and item not in branches:
                print(f'Missing std heading -- {df.at["Congress", "Values"]}, {df.at["Session", "Values"]}: {item}')
                errors_found = True
                
if not errors_found:
    print('No errors found.')

No errors found.


***
<font color='blue'>**Note:**</font>  
Any OCR errors found were updated directly in the Excel files.

***
## Merge Data
***

In [27]:
# Create dataframe for merged data
conf_df = pd.DataFrame()

In [28]:
# Transpose the dataframe read from each file and add to the general activity dataframe
for df in conf_list:
    df = df.transpose().reset_index()
    conf_df = pd.concat([conf_df, df], ignore_index=True)

In [29]:
# Preview dataframe
conf_df.head()

,index,Congress,Session,Start Date,End Date,Civilian,"Civilian, New nominations","Civilian, Confirmed","Civilian, Unconfirmed","Civilian, Withdrawn",...,Total failed at November sine die adjournment,"Civilian, Recess reappointments",Total recess reappointments,"Civilian, Superseded by recess reappointments",Total superseded by recess reappointments,Civilian lists,"Civilian lists, New nominations","Civilian lists, Carryover nominations","Civilian lists, Confirmed","Civilian lists, Unconfirmed"
0,Values,100,1,1987-01-06 00:00:00,1987-12-22 00:00:00,NaN,470,331,112,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Values,100,2,1988-01-25 00:00:00,1988-10-22 00:00:00,NaN,404,335,170,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Values,101,1,1989-01-03 00:00:00,1989-11-22 00:00:00,NaN,586,422,79,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Values,101,2,1990-01-23 00:00:00,1990-10-28 00:00:00,NaN,435,430,73,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Values,102,1,1991-01-03 00:00:00,1992-01-03 00:00:00,NaN,536,420,98,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# Drop the Index column
conf_df.drop(columns='index', inplace=True)

***
## Delete Headings
***

In [31]:
# Delete section headings, which are not needed at this stage
conf_df.drop(columns=branches, inplace=True)

***
## Combine Civilian Categories
***

In [32]:
# List all columns that contain Civilian
conf_df[conf_df.columns[conf_df.columns.str.contains('Civilian')]].columns

Index(['Civilian, New nominations', 'Civilian, Confirmed',
       'Civilian, Unconfirmed', 'Civilian, Withdrawn', 'Civilian, Rejected',
       'Civilian, Returned', 'Civilian (lists), New nominations',
       'Civilian (lists), Confirmed', 'Civilian (lists), Unconfirmed',
       'Civilian, Carryover nominations',
       'Civilian (lists), Carryover nominations',
       'Civilian, Failed at Aug adjournment',
       'Civilian, Failed at sine die adjournment',
       'Civilian (lists), Failed at sine die adjournment',
       'Civilian, Failed at August-September adjournment',
       'Civilian, Failed at adjournment',
       'Civilian (FS, PHS, CG, NOAA), New nominations',
       'Civilian (FS, PHS, CG, NOAA), Confirmed',
       'Civilian (FS, PHS, CG, NOAA), Unconfirmed',
       'Civilian (FS, PHS, CG, NOAA), Withdrawn',
       'Civilian (FS, PHS, CG, NOAA), Carryover nominations',
       'Civilian, Returned at sine die adjournment',
       'Civilian (other than lists), New nominations',


In [33]:
# Since Civilian nominations have been listed using a variety of headings over the years we will simplify all to simply "Civilian"
civ_cols = conf_df[conf_df.columns[conf_df.columns.str.contains('Civilian')]].columns.to_list()
for label in labels:
    cols_to_merge = [x for x in civ_cols if label == x[-len(label):]]
    if len(cols_to_merge) > 1:
        sum_col = 'Civilian, ' + label
        conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
        cols_to_merge.remove(sum_col)
        conf_df.drop(columns=cols_to_merge, inplace=True)

In [34]:
# Review updated Civilian columns headings
print(conf_df[conf_df.columns[conf_df.columns.str.contains('Civilian')]].columns)

Index(['Civilian, New nominations', 'Civilian, Confirmed',
       'Civilian, Unconfirmed', 'Civilian, Withdrawn', 'Civilian, Rejected',
       'Civilian, Returned', 'Civilian, Carryover nominations',
       'Civilian, Failed at Aug adjournment',
       'Civilian, Failed at sine die adjournment',
       'Civilian, Failed at August-September adjournment',
       'Civilian, Failed at adjournment',
       'Civilian, Returned at sine die adjournment',
       'Civilian, Returned to White House',
       'Civilian, Failed at November sine die adjournment',
       'Civilian, Recess reappointments',
       'Civilian, Superseded by recess reappointments'],
      dtype='object')


In [35]:
# Update branches list for accuracy
branches = [x for x in branches if 'Civilian' not in x or x == 'Civilian']
print(branches)

['Air Force', 'Army', 'Civilian', 'Marine Corps', 'Navy', 'Space Force']


***
## Combine Failed Dispositions
***

In [36]:
# List all column headings that contain "Failed"
conf_df[conf_df.columns[conf_df.columns.str.contains('Failed')]].columns

Index(['Civilian, Failed at Aug adjournment',
       'Civilian, Failed at sine die adjournment',
       'Army, Failed at sine die adjournment',
       'Civilian, Failed at August-September adjournment',
       'Civilian, Failed at adjournment',
       'Army, Failed at August-September adjournment',
       'Army, Failed at November sine die adjournment',
       'Air Force, Failed at August-September adjournment',
       'Civilian, Failed at November sine die adjournment'],
      dtype='object')

In [37]:
# Simplify all variations of "failed" to simply "Failed"
failed_cols = conf_df[conf_df.columns[conf_df.columns.str.contains('Failed')]].columns.to_list()
for branch in branches:
    cols_to_merge = [x for x in failed_cols if x.startswith(branch)]
    if len(cols_to_merge) > 1:
        sum_col = branch + ', Failed'
        conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
        if sum_col in cols_to_merge:
            cols_to_merge.remove(sum_col)
        conf_df.drop(columns=cols_to_merge, inplace=True)
    elif len(cols_to_merge) == 1:
        conf_df.rename(columns={cols_to_merge[0]: branch + ', Failed'}, inplace=True)

In [38]:
# Review updated Failed column headings
print(conf_df[conf_df.columns[conf_df.columns.str.contains('Failed')]].columns)

Index(['Air Force, Failed', 'Army, Failed', 'Civilian, Failed'], dtype='object')


In [39]:
# Update labels list for accuracy
labels = [x for x in labels if 'Failed' not in x]
labels.append('Failed')
labels

['Carryover nominations',
 'Confirmed',
 'New nominations',
 'Recess reappointments',
 'Rejected',
 'Returned',
 'Returned at sine die adjournment',
 'Returned to White House',
 'Superseded by recess reappointments',
 'Unconfirmed',
 'Withdrawn',
 'Failed']

***
## Combine Returned Dispositions
***

In [40]:
# List all column headings that contain "returned"
conf_df[conf_df.columns[conf_df.columns.str.contains('Returned')]].columns

Index(['Civilian, Returned', 'Air Force, Returned', 'Army, Returned',
       'Marine Corps, Returned', 'Civilian, Returned at sine die adjournment',
       'Navy, Returned at sine die adjournment',
       'Civilian, Returned to White House',
       'Air Force, Returned to White House', 'Army, Returned to White House',
       'Navy, Returned to White House',
       'Marine Corps, Returned to White House'],
      dtype='object')

In [41]:
# Simplify all variations of "returned" to simply "Returned"
returned_cols = conf_df[conf_df.columns[conf_df.columns.str.contains('Returned')]].columns.to_list()
for branch in branches:
    cols_to_merge = [x for x in returned_cols if x.startswith(branch)]
    if len(cols_to_merge) > 1:
        sum_col = branch + ', Returned'
        conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
        if sum_col in cols_to_merge:
            cols_to_merge.remove(sum_col)
        conf_df.drop(columns=cols_to_merge, inplace=True)
    elif len(cols_to_merge) == 1:
        conf_df.rename(columns={cols_to_merge[0]: branch + ', Returned'}, inplace=True)

In [42]:
# Review updated Returned column headings
print(conf_df[conf_df.columns[conf_df.columns.str.contains('Returned')]].columns)

Index(['Civilian, Returned', 'Air Force, Returned', 'Army, Returned',
       'Marine Corps, Returned', 'Navy, Returned'],
      dtype='object')


In [43]:
# Update labels list for accuracy
labels = [x for x in labels if 'Returned' not in x]
labels.append('Returned')
labels

['Carryover nominations',
 'Confirmed',
 'New nominations',
 'Recess reappointments',
 'Rejected',
 'Superseded by recess reappointments',
 'Unconfirmed',
 'Withdrawn',
 'Failed',
 'Returned']

***
## Combine Recess Reappointments
***

In [44]:
# List all column headings that contain "reappointment"
conf_df[conf_df.columns[conf_df.columns.str.contains('reappointment')]].columns

Index(['Civilian, Recess reappointments', 'Total recess reappointments',
       'Civilian, Superseded by recess reappointments',
       'Total superseded by recess reappointments'],
      dtype='object')

In [45]:
# Simplify all variations of "reappointment" to simply "Recess reappointments"
reappt_cols = conf_df[conf_df.columns[conf_df.columns.str.contains('reappointment')]].columns.to_list()
reappt_cols.remove('Total superseded by recess reappointments')
for branch in branches:
    cols_to_merge = [x for x in reappt_cols if x.startswith(branch)]
    if len(cols_to_merge) > 1:
        sum_col = branch + ', Recess reappointments'
        conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
        if sum_col in cols_to_merge:
            cols_to_merge.remove(sum_col)
        conf_df.drop(columns=cols_to_merge, inplace=True)
    elif len(cols_to_merge) == 1:
        conf_df.rename(columns={cols_to_merge[0]: branch + ', Recess reappointments'}, inplace=True)

In [46]:
# Review updated Reappointment column headings
print(conf_df[conf_df.columns[conf_df.columns.str.contains('reappointment')]].columns)

Index(['Civilian, Recess reappointments', 'Total recess reappointments',
       'Total superseded by recess reappointments'],
      dtype='object')


In [47]:
# Update labels list for accuracy
labels = [x for x in labels if 'reappointment' not in x]
labels.append('Recess reappointments')
labels

['Carryover nominations',
 'Confirmed',
 'New nominations',
 'Rejected',
 'Unconfirmed',
 'Withdrawn',
 'Failed',
 'Returned',
 'Recess reappointments']

***
## Clean up Total Columns
***

In [48]:
# Revew total column headings
conf_df[conf_df.columns[conf_df.columns.str.contains('Total')]].columns

Index(['Total new nominations', 'Total confirmed', 'Total unconfirmed',
       'Total withdrawn', 'Total rejected', 'Total returned',
       'Total carryover nominations', 'Total failed at Aug adjournment',
       'Total failed at sine die adjournment',
       'Total failed at August-September adjournment',
       'Total failed at adjournment', 'Total returned at sine die adjournment',
       'Total returned to the White House',
       'Total failed at November sine die adjournment',
       'Total recess reappointments',
       'Total superseded by recess reappointments'],
      dtype='object')

In [49]:
# Combine failed total columns
cols_to_merge = [x for x in std_headings if x.startswith('Total ') and 'failed' in x]
sum_col = 'Total failed'
conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
conf_df.drop(columns=cols_to_merge, inplace=True)

In [50]:
# Combine returned total columns
cols_to_merge = [x for x in std_headings if x.startswith('Total ') and 'returned' in x]
sum_col = 'Total returned'
conf_df[sum_col] = conf_df[cols_to_merge].sum(axis=1)
cols_to_merge.remove(sum_col)
conf_df.drop(columns=cols_to_merge, inplace=True)

In [51]:
# Combine recess reappointments
cols_to_merge = ['Total recess reappointments', 'Total superseded by recess reappointments']
conf_df['Total recess reappointments'] = conf_df[cols_to_merge].sum(axis=1)
conf_df.drop(columns='Total superseded by recess reappointments', inplace=True)

In [52]:
# Revew total column headings
conf_df[conf_df.columns[conf_df.columns.str.contains('Total')]].columns

Index(['Total new nominations', 'Total confirmed', 'Total unconfirmed',
       'Total withdrawn', 'Total rejected', 'Total returned',
       'Total carryover nominations', 'Total recess reappointments',
       'Total failed'],
      dtype='object')

In [53]:
# Preview updated dataframe
conf_df.head()

,Congress,Session,Start Date,End Date,"Civilian, New nominations","Civilian, Confirmed","Civilian, Unconfirmed","Civilian, Withdrawn","Civilian, Rejected","Civilian, Returned",...,"Space Force, Unconfirmed","Space Force, Withdrawn","Space Force, Carryover nominations","Air Force, Failed","Civilian, Recess reappointments",Total recess reappointments,"Army, Failed","Civilian, Failed","Navy, Returned",Total failed
0,100,1,1987-01-06 00:00:00,1987-12-22 00:00:00,4718,4569,122,10,1,16,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,100,2,1988-01-25 00:00:00,1988-10-22 00:00:00,2670,2357,424,11,NaN,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,101,1,1989-01-03 00:00:00,1989-11-22 00:00:00,3564,3399,79,34,1,0,...,NaN,NaN,NaN,NaN,0,0,1,51,0,52
3,101,2,1990-01-23 00:00:00,1990-10-28 00:00:00,2364,2145,287,11,NaN,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,102,1,1991-01-03 00:00:00,1992-01-03 00:00:00,3376,3256,101,9,NaN,0,...,NaN,NaN,NaN,NaN,0,0,0,10,0,10


***
## Validate Datatypes
***

In [54]:
# Review current datatypes
conf_df.dtypes

Congress                               object
Session                                object
Start Date                             object
End Date                               object
Civilian, New nominations              object
Civilian, Confirmed                    object
Civilian, Unconfirmed                  object
Civilian, Withdrawn                    object
Civilian, Rejected                     object
Civilian, Returned                     object
Air Force, New nominations             object
Air Force, Confirmed                   object
Air Force, Unconfirmed                 object
Air Force, Returned                    object
Army, New nominations                  object
Army, Confirmed                        object
Army, Unconfirmed                      object
Army, Returned                         object
Navy, New nominations                  object
Navy, Confirmed                        object
Navy, Unconfirmed                      object
Marine Corps, New nominations     

In [55]:
# Infer datatypes
conf_df = conf_df.infer_objects()

In [56]:
# Review updated datatypes
conf_df.dtypes

Congress                                        int64
Session                                         int64
Start Date                             datetime64[ns]
End Date                               datetime64[ns]
Civilian, New nominations                       int64
Civilian, Confirmed                             int64
Civilian, Unconfirmed                           int64
Civilian, Withdrawn                             int64
Civilian, Rejected                            float64
Civilian, Returned                              int64
Air Force, New nominations                      int64
Air Force, Confirmed                            int64
Air Force, Unconfirmed                        float64
Air Force, Returned                             int64
Army, New nominations                           int64
Army, Confirmed                                 int64
Army, Unconfirmed                             float64
Army, Returned                                  int64
Navy, New nominations       

In [57]:
# Given the length of the list, check specifically for object datatypes
conf_df.select_dtypes('object').columns

Index([], dtype='object')

***
<font color='blue'>**Note:**</font>  
Any numeric fields that did not convert to float were examined for OCR issues. These issues were corrected in the Excel files.

***
## Sort Dataframe
***

In [58]:
# Sort data by Congress and Session
conf_df.sort_values(['Congress', 'Session'], inplace=True)
conf_df.reset_index(drop=True, inplace=True)
conf_df.head()

,Congress,Session,Start Date,End Date,"Civilian, New nominations","Civilian, Confirmed","Civilian, Unconfirmed","Civilian, Withdrawn","Civilian, Rejected","Civilian, Returned",...,"Space Force, Unconfirmed","Space Force, Withdrawn","Space Force, Carryover nominations","Air Force, Failed","Civilian, Recess reappointments",Total recess reappointments,"Army, Failed","Civilian, Failed","Navy, Returned",Total failed
0,98,1,1983-01-03,1983-11-18,3454,2978,0,2,NaN,0,...,NaN,NaN,NaN,1.0,0,0,2,474,0,477
1,98,2,1984-01-23,1984-10-12,4127,4001,107,2,NaN,0,...,NaN,NaN,NaN,NaN,17,17,0,0,0,0
2,99,1,1985-01-03,1985-12-20,3719,3603,69,7,NaN,0,...,NaN,NaN,NaN,NaN,6,6,0,34,0,34
3,99,2,1986-01-21,1986-10-18,2046,2037,70,8,NaN,0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,100,1,1987-01-06,1987-12-22,4718,4569,122,10,1.0,16,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


***
## Write to Excel
***

In [59]:
conf_df.to_excel('../data/confirmation_data.xlsx', index=False)

***
**End**
***